# ACCESS to Hooktheory API

In [39]:
import requests
import pandas as pd
import time


Necesitamos unas credenciales para acceder a la API

In [40]:
def get_auth_key(username, password):
    auth_url = "https://api.hooktheory.com/v1/users/auth"
    auth_data = {
        "username": username,
        "password": password
    }
    
    # Solicitar token de autenticación
    auth_response = requests.post(auth_url, json=auth_data)
    
    if auth_response.status_code == 200:
        return auth_response.json()["activkey"]
    else:
        raise Exception(f"Error en la autenticación: {auth_response.status_code} - {auth_response.text}")
    

my_key = get_auth_key("ivantoribio", "MusicIsForEveryone12")

my_key

'e02a2d8962a0c70fc763994e9fb41f02'

Creamos una función para conseguir todas las canciones en la base de datos que siguen una determinada progresión

In [41]:
def get_songs_by_progression(auth_key, progression):
    songs_url = f"https://api.hooktheory.com/v1/trends/songs?cp={progression}"
    
    headers = {
        "Authorization": f"Bearer {auth_key}",
        "Accept": "application/json",
        "Content-Type": "application/json"
    }
    
    songs_data = []
    page = 1
    
    while page <= 5:
        # Añadimos paginación en caso de que haya muchas canciones
        response = requests.get(songs_url + f"&page={page}", headers=headers)
        
        if response.status_code != 200:
            raise Exception(f"Error al obtener canciones: {response.status_code} - {response.text}")
        
        songs = response.json()
        
        if not songs:
            break  # Salimos si no hay más canciones en la siguiente página

        # Agregamos cada canción a nuestra lista
        for song in songs:
            songs_data.append({
                "artist": song["artist"],
                "song": song["song"],
                "section": song["section"],
                "url": song["url"],
                "progression": progression
            })
        
        time.sleep(1)
        page += 1  # Avanzamos a la siguiente página

    return songs_data

get_songs_by_progression(my_key, "4,1")

[{'artist': 'Adele',
  'song': 'Someone Like You',
  'section': 'Chorus',
  'url': 'http://www.hooktheory.com/theorytab/view/adele/someone-like-you#chorus',
  'progression': '4,1'},
 {'artist': 'Adele',
  'song': 'Someone Like You',
  'section': 'Verse',
  'url': 'http://www.hooktheory.com/theorytab/view/adele/someone-like-you#verse',
  'progression': '4,1'},
 {'artist': 'Aerosmith',
  'song': "Cryin'",
  'section': 'Pre-Chorus',
  'url': 'http://www.hooktheory.com/theorytab/view/aerosmith/cryin#pre-chorus',
  'progression': '4,1'},
 {'artist': 'Aerosmith',
  'song': "Cryin'",
  'section': 'Verse',
  'url': 'http://www.hooktheory.com/theorytab/view/aerosmith/cryin#verse',
  'progression': '4,1'},
 {'artist': 'Alt-J',
  'song': 'Something Good',
  'section': 'Verse',
  'url': 'http://www.hooktheory.com/theorytab/view/alt-j/something-good#verse',
  'progression': '4,1'},
 {'artist': 'Augustana',
  'song': 'Boston',
  'section': 'Verse',
  'url': 'http://www.hooktheory.com/theorytab/view/

Existe un límite de peticiones que podemos hacer a la API. Ese límite es de 10 requests por cada 10 segundos. 

In [42]:
# Hacer una solicitud para obtener datos de la API (por ejemplo, tendencias de nodos)
headers = {
        "Authorization": f"Bearer {my_key}",
        "Accept": "application/json",
        "Content-Type": "application/json"
    }
url = "https://api.hooktheory.com/v1/trends/nodes"
response = requests.get(url, headers=headers)

# Verificar el estado de la respuesta
if response.status_code == 200:
    # Obtener los encabezados de límite de tasa
    rate_limit_limit = response.headers.get("X-Rate-Limit-Limit")
    rate_limit_remaining = response.headers.get("X-Rate-Limit-Remaining")
    rate_limit_reset = response.headers.get("X-Rate-Limit-Reset")

    # Mostrar los valores obtenidos
    print("X-Rate-Limit-Limit:", rate_limit_limit)
    print("X-Rate-Limit-Remaining:", rate_limit_remaining)
    print("X-Rate-Limit-Reset (en segundos):", rate_limit_reset)
else:
    print("Error en la solicitud:", response.status_code, response.text)

X-Rate-Limit-Limit: 10
X-Rate-Limit-Remaining: 8
X-Rate-Limit-Reset (en segundos): 2


In [44]:
def get_progressions(auth_key):
    chords_url = "https://api.hooktheory.com/v1/trends/nodes"
    headers = {
        "Authorization": f"Bearer {auth_key}",
        "Accept": "application/json",
        "Content-Type": "application/json"
    }
    
    chords_response = requests.get(chords_url, headers=headers)
    if chords_response.status_code == 200:
        progressions = [node["child_path"] for node in chords_response.json()]
    else:
        raise Exception(f"Error al obtener acordes: {chords_response.status_code} - {chords_response.text}")

    
    # Limitamos a las primeras progresiones para no exceder el rate limit
    songs_data = []
    for progression in progressions[:5]:  # Cambia el límite si necesitas más progresiones
        print(f"Obteniendo canciones para la progresión: {progression}")
        songs_data.extend(get_songs_by_progression(auth_key, progression))
    
    return songs_data


# Convertimos a DataFrame de pandas
songs_df = pd.DataFrame(get_progressions(my_key))
print(songs_df.head())


Obteniendo canciones para la progresión: 1
Obteniendo canciones para la progresión: 4
Obteniendo canciones para la progresión: 6


Exception: Error al obtener canciones: 500 - {"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}

La idea es conseguir todas las combinaciones de 4 acordes posibles y capturar las canciones que tienen esas progresiones para almacenar los datos y poder tratarlos en un dataset de pandas.

In [48]:
import itertools

# Definir los acordes posibles (por ejemplo, del I al VII)
possible_chords = ["1", "2", "3", "4", "5", "6", "7"]

#Definimos cuando una combinacion es válida. Por ejemplo, (1,1,2,4) no lo es pero (1,2,3,1) sí lo es.
def is_valid(combination):
    for i in range(len(combination) - 1):
        if combination[i] == combination[i + 1]:
            return False
    return True


# Generar todas las combinaciones de cuatro acordes posibles
combinations = [combination for combination in itertools.product(possible_chords, repeat=4) if is_valid(combination)]

# Crear DataFrame para almacenar los datos
df_canciones = pd.DataFrame(columns=["cp", "artist", "song", "section", "url"])

# Encabezado para la autenticación (reemplaza "tu_token" por tu activkey)
headers = {
    "Authorization": f"Bearer {my_key}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Función para obtener las canciones para una combinación de acordes
def obtener_canciones(cp):
    url = f"https://api.hooktheory.com/v1/trends/songs?cp={','.join(cp)}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 429:  # Código 429: demasiado tráfico
        wait_time = int(response.headers.get("X-Rate-Limit-Reset", 10))
        print(f"Esperando {wait_time} segundos debido al límite de tasa")
        time.sleep(wait_time)
        return obtener_canciones(cp)  # Reintentar después de esperar
    else:
        print(f"Error {response.status_code}: {response.json()}")
        return []
    
            
# Iterar sobre cada combinación y hacer la solicitud
for cp in combinations:
    canciones = obtener_canciones(cp)
    if canciones:  # Si hay canciones en la respuesta
        for cancion in canciones:
            df_canciones = df_canciones._append({
                "cp": ','.join(cp),
                "artist": cancion["artist"],
                "song": cancion["song"],
                "section": cancion["section"],
                "url": cancion["url"]
            }, ignore_index=True)
    time.sleep(1)  # Pausa de 1 segundo para evitar sobrecarga de solicitudes

Error 500: {'name': 'Internal Server Error', 'message': 'An internal server error occurred.', 'code': 0, 'status': 500}


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:




"""
# Definimos el encabezado de autenticación
headers = {
    "Authorization": f"Bearer {my_ath_key}",
    "Accept": "application/json",
    "Content-Type": "application/json" # Especificamos que queremos los datos en formato JSON
}

# Realizamos una solicitud GET para obtener los acordes
chords_response = requests.get(chords_url, headers=headers)

# Verificamos si la solicitud fue exitosa
if chords_response.status_code == 200:
    chords_data = chords_response.json()
    print("Acordes y probabilidades de progresión:", chords_data)
else:
    print("Error al obtener los acordes:", chords_response.status_code, chords_response.text)
"""